In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from collections import OrderedDict
from tqdm import tqdm
import datetime

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
num = 0
for filename in os.listdir('D:/data2'):
    entries = os.listdir('D:/data2/' + filename)
    # print(filename)
    num += len(entries)
print(num)

288


In [3]:
dataset = datasets.ImageFolder('D:/data2', transform=transforms.Compose([
        transforms.Resize((644, 644)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))
class_names = dataset.classes
len(class_names)

7

In [4]:
trainds, valds, testds = torch.utils.data.random_split(dataset, [0.4, 0.3, 0.3], generator=torch.Generator().manual_seed(15275))

dataloaders = {'train': torch.utils.data.DataLoader(trainds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'val': torch.utils.data.DataLoader(valds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'test': torch.utils.data.DataLoader(testds, batch_size=32,
                                             shuffle=True, num_workers=16)}

dataset_sizes = {'train': len(trainds), 
               'val': len(valds), 
               'test': len(testds)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []


    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)
            print(datetime.datetime.now())

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tqdm(dataloaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]
                if phase == 'train':
                    train_loss.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    valid_loss.append(epoch_loss)
                    valid_acc.append(epoch_acc)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model, train_loss, train_acc, valid_loss, valid_acc

In [12]:
model_ft = models.mobilenet_v2(weights='IMAGENET1K_V1')
# Here the size of each output sample is set to 5.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.classifier[1] = nn.Linear(1280,len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [13]:
model_ft.features

Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (

In [14]:
model_ft

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [17]:
for name, param in model_ft.named_parameters():
    if param.requires_grad:
        print(f"{name} will be updated.")
    else:
        print(f"{name} will not be updated.")

features.0.0.weight will be updated.
features.0.1.weight will be updated.
features.0.1.bias will be updated.
features.1.conv.0.0.weight will be updated.
features.1.conv.0.1.weight will be updated.
features.1.conv.0.1.bias will be updated.
features.1.conv.1.weight will be updated.
features.1.conv.2.weight will be updated.
features.1.conv.2.bias will be updated.
features.2.conv.0.0.weight will be updated.
features.2.conv.0.1.weight will be updated.
features.2.conv.0.1.bias will be updated.
features.2.conv.1.0.weight will be updated.
features.2.conv.1.1.weight will be updated.
features.2.conv.1.1.bias will be updated.
features.2.conv.2.weight will be updated.
features.2.conv.3.weight will be updated.
features.2.conv.3.bias will be updated.
features.3.conv.0.0.weight will be updated.
features.3.conv.0.1.weight will be updated.
features.3.conv.0.1.bias will be updated.
features.3.conv.1.0.weight will be updated.
features.3.conv.1.1.weight will be updated.
features.3.conv.1.1.bias will be up

In [18]:
model_ft, train_loss, train_acc, valid_loss, valid_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=70)

Epoch 0/69
----------
2025-03-02 20:32:54.500752


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [05:34<00:00, 83.65s/it]


train Loss: 1.1228 Acc: 0.5345


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:14<00:00, 24.98s/it]


val Loss: 1.1490 Acc: 0.5814

Epoch 1/69
----------
2025-03-02 20:39:44.146420


...

val Loss: 0.2726 Acc: 0.9651

Epoch 69/69
----------
2025-03-03 04:12:37.373129


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [05:29<00:00, 82.30s/it]


train Loss: 0.0037 Acc: 1.0000


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:11<00:00, 23.72s/it]


val Loss: 0.2692 Acc: 0.9651

Training complete in 466m 23s
Best val Acc: 0.965116


In [19]:
def test_model(model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(testds)
    correct = 0.0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloaders['test']:
            X= X.to(device)
            outputs = model(X)
            _, preds = torch.max(outputs, 1)
            # print('===================')
            # print(preds)
            # print(y.data)
            correct += torch.sum(preds == y.data)

    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%")

test_model(model_ft, criterion)

Test Error: 
 Accuracy: 93.0%
